In [4]:
URL = "https://sametinget.kommunetv.no/api/streams?streamType=1&id=244"

In [5]:
import requests

request = requests.get(URL)

In [6]:
assert request.status_code == 200

In [8]:
import json
streams = json.loads(request.text)

In [9]:
streams

{'stream': {'assetId': 1776,
  'assetCopyrightText': None,
  'attachments': [],
  'availableFrom': None,
  'availableTo': None,
  'categoryId': 16,
  'description': '<p>Plenumsm&oslash;tet starter tirsdag 3. juni kl. 09.00 og avsluttes fredag 6. juni innen kl. 10.00. Se saksliste og program:&nbsp;https://sametinget.no/Kalender/CalendarEvent.aspx?Id=1494&amp;MId1=7</p>',
  'id': 244,
  'imageUrl': 'https://hub.aventia.no/resources/defaults/default-movie.png',
  'publishDate': '2025-06-05T06:30:00Z',
  'streamType': 1,
  'title': 'Direktesending - Sametingets plenum 05.06.2025',
  'tupleId': 487,
  'views': 654,
  'voting': True,
  'externalCode': None,
  'movieConfiguration': None,
  'hidden': False,
  'allowQuestionForm': False,
  'likes': 0,
  'dislikes': 0},
 'bookmarks': [{'assetId': None,
   'documents': [{'id': 3256,
     'title': '50586',
     'downloadUrl': 'https://innsyn.onacos.no/sametinget/mote/norsk/wfinnsyn.ashx?response=journalpost_detaljer&journalpostid=2021190422',
    